# A Tiny Temperature Converter Library
## Written in Python (a naive approach)

Based on a wonderful example from _Erik Engheim_ 

Start with a simple class (you should almost never start with a class, but that's not the point now ;-)

In [1]:
class Celsius(object):
    def __init__(self, temp):
        if type(temp) == float or type(temp) == int:
            self.value = temp
        else:
            self.value = temp.to_celsius().value

    def __repr__(self):
        return "Celsius({0})".format(self.value)
        
    def to_celsius(self):
        return self
 
    def to_kelvin(self):
        return Kelvin(self.value + 273.15)
        
    def __add__(self, temp):
        return Celsius(self.value + temp.to_celsius().value)
        
    def __sub__(self, temp):
        return Celsius(self.value - temp.to_celsius().value)

In [2]:
Celsius(42)

Celsius(42)

In [3]:
Celsius(42) + Celsius(23)

Celsius(65)

In [4]:
Celsius(5) - Celsius(23)

Celsius(-18)

Implement the second temperature unit...

In [5]:
class Kelvin(object):
    def __init__(self, temp):
        if type(temp) == float or type(temp) == int:
            self.value = temp
        else:
            self.value = temp.to_kelvin().value
            
    def __repr__(self):
        return "Kelvin({0})".format(self.value)
        
    def to_kelvin(self):
        return self

    def to_celsius(self):
        return Kelvin(self.value - 273.15)
        
    def __add__(self, temp):
        return Kelvin(self.value + temp.to_kelvin().value)
        
    def __sub__(self, temp):
        return Kelvin(self.value - temp.to_kelvin().value)

In [6]:
Kelvin(5)

Kelvin(5)

In [7]:
Kelvin(5) + Kelvin(23)

Kelvin(28)

In [8]:
Kelvin(5) + Celsius(5)

Kelvin(283.15)

# What about Fahrenheit?

Of course, we are violating the open/closed SOLID principle, but if we do it the right way, it will get even worse with the performance in Python...

### We need to modify the existing `Celsius` and `Kelvin` implementation and add a new class `Fahrenheit`.

In [9]:
class Celsius(object):
    def __init__(self, temp):
        if type(temp) == float or type(temp) == int:
            self.value = temp
        else:
            self.value = temp.to_celsius().value

    def __repr__(self):
        return "Celsius({0})".format(self.value)
        
    def to_celsius(self):
        return self
 
    def to_kelvin(self):
        return Kelvin(self.value + 273.15)
    
    def to_fahrenheit(self):  # our new method
        return Fahrenheit(self.value * 1.8 + 32)
        
    def __add__(self, temp):
        return Celsius(self.value + temp.to_celsius().value)
        
    def __sub__(self, temp):
        return Celsius(self.value - temp.to_celsius().value)

In [10]:
class Kelvin(object):
    def __init__(self, temp):
        if type(temp) == float or type(temp) == int:
            self.value = temp
        else:
            self.value = temp.to_kelvin().value
            
    def __repr__(self):
        return "Kelvin({0})".format(self.value)
        
    def to_kelvin(self):
        return self

    def to_celsius(self):
        return Kelvin(self.value - 273.15)

    def to_fahrenheit(self):
        return Fahrenheit(self.value * 1.8 - 459.67)
    
    def __add__(self, temp):
        return Kelvin(self.value + temp.to_kelvin().value)
        
    def __sub__(self, temp):
        return Kelvin(self.value - temp.to_kelvin().value)      

In [11]:
class Fahrenheit(object):
    def __init__(self, temp):
        if type(temp) == float or type(temp) == int:
            self.value = temp
        else:
            self.value = temp.to_fahrenheit().value
            
    def __repr__(self):
        return "Fahrenheit({0})".format(self.value)
        
    def to_kelvin(self):
        return Kelvin((self.value + 459.67) / 1.8)

    def to_celsius(self):
        return Celsius((self.value - 32) / 1.8)

    def to_fahrenheit(self):
        return self
    
    def __add__(self, temp):
        return Fahrenheit(self.value + temp.to_fahrenheit().value)
        
    def __sub__(self, temp):
        return Fahrenheit(self.value - temp.to_fahrenheit().value)  

Now we can do fancy things like adding three different units and convert them to `Kelvin`:

In [12]:
Fahrenheit(23) + Celsius(42) - Kelvin(5)  # this does not need to make sense now ;)

Fahrenheit(581.27)

In [13]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

Fahrenheit(-410.27000000000004)

## Alright, let's add the temperature units `Rankine` and `Réaumur`!

Just kidding...

Although the overall design is utterly ugly at its best, a similar approach written in C++ or Java will be optimised quite well with a decent compiler.

This is probably also the reason why you see this kind of software design in many many places. But that's another story.

## This will be **slow as hell** in Python!

A better approach would be e.g. writing `numpy.ufunc`s, trying to get the data into homogeneous arrays and then vectorise as much as possible.

### Alright, let's have a look behind the scenes...

We wrap the calculation from the previous slide into a function:

In [14]:
def example_calculation():
    return Fahrenheit(23) + Celsius(42) - Kelvin(5)

...and disassemble it:

In [15]:
import dis
dis.dis(example_calculation)

  2           0 LOAD_GLOBAL              0 (Fahrenheit)
              2 LOAD_CONST               1 (23)
              4 CALL_FUNCTION            1
              6 LOAD_GLOBAL              1 (Celsius)
              8 LOAD_CONST               2 (42)
             10 CALL_FUNCTION            1
             12 BINARY_ADD
             14 LOAD_GLOBAL              2 (Kelvin)
             16 LOAD_CONST               3 (5)
             18 CALL_FUNCTION            1
             20 BINARY_SUBTRACT
             22 RETURN_VALUE


The initialisation of `Fahrenheit` is already a bunch of loads and function calls:

In [16]:
dis.dis(Fahrenheit.__init__)

  3           0 LOAD_GLOBAL              0 (type)
              2 LOAD_FAST                1 (temp)
              4 CALL_FUNCTION            1
              6 LOAD_GLOBAL              1 (float)
              8 COMPARE_OP               2 (==)
             10 POP_JUMP_IF_TRUE        24
             12 LOAD_GLOBAL              0 (type)
             14 LOAD_FAST                1 (temp)
             16 CALL_FUNCTION            1
             18 LOAD_GLOBAL              2 (int)
             20 COMPARE_OP               2 (==)
             22 POP_JUMP_IF_FALSE       32

  4     >>   24 LOAD_FAST                1 (temp)
             26 LOAD_FAST                0 (self)
             28 STORE_ATTR               3 (value)
             30 JUMP_FORWARD            12 (to 44)

  6     >>   32 LOAD_FAST                1 (temp)
             34 LOAD_METHOD              4 (to_fahrenheit)
             36 CALL_METHOD              0
             38 LOAD_ATTR                3 (value)
             40 LOAD_FAST

### And keep in mind, this is only Python byte code which will be blown up to a massive amount of machine code...

## What about the performance?

In [17]:
def example_calculation():
    return Fahrenheit(23) + Celsius(42) - Kelvin(5)

In [18]:
%timeit example_calculation()

3.34 µs ± 45 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [19]:
def another_calculation(a, b, c):
    return Fahrenheit(a) + Celsius(b) - Kelvin(c)

In [20]:
%timeit another_calculation(1, 2, 3)

3.4 µs ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
